## BLE RSSI Dataset for Indoor localization

The dataset was created using the RSSI readings of an array of 13 ibeacons in the first floor of Waldo Library, Western Michigan University. Data was collected using iPhone 6S. The dataset contains two sub-datasets: a labeled dataset (1420 instances) and an unlabeled dataset (5191 instances). The recording was performed during the operational hours of the library. For the labeled dataset, the input data contains the location (label column), a timestamp, followed by RSSI readings of 13 iBeacons. RSSI measurements are negative values. Bigger RSSI values indicate closer proximity to a given iBeacon (e.g., RSSI of -65 represent a closer distance to a given iBeacon compared to RSSI of -85). For out-of-range iBeacons, the RSSI is indicated by -200. The locations related to RSSI readings are combined in one column consisting a letter for the column and a number for the row of the position. The following figure depicts the layout of the iBeacons as well as the arrange of locations.

## Clone git repo

In [1]:
BRANCH_NAME="dev" #Provide git branch name "master" or "dev"
! git clone -b $BRANCH_NAME https://github.com/CiscoAI/cisco-kubeflow-starter-pack.git

Cloning into 'cisco-kubeflow-starter-pack'...
remote: Enumerating objects: 522, done.
remote: Counting objects: 100% (522/522), done.
remote: Compressing objects: 100% (310/310), done.
remote: Total 2577 (delta 198), reused 403 (delta 113), pack-reused 2055
Receiving objects: 100% (2577/2577), 7.63 MiB | 32.82 MiB/s, done.
Resolving deltas: 100% (1002/1002), done.


# Configure docker credentials

Get your docker registry user and password encoded in base64 

echo -n USER:PASSWORD | base64 

Create a config.json file with your Docker registry url and the previous generated base64 string 

In [2]:
!echo -n USER:PASSWORD | base64

VVNFUjpQQVNTV09SRA==


In [3]:
%%writefile config.json
{
    "auths": {
        "https://index.docker.io/v1/": {
            "auth": "<<Provide previous generated base64 string>>"
        }
    }
}

Writing config.json


## Create requirements.txt

In [4]:
%%writefile requirements.txt
pandas
numpy
scikit-learn>=0.21.0,<0.22
seldon-core
tornado>=6.0.3
kubeflow-tfjob
kubeflow-fairing
tensorflow==1.14.0
cloudpickle==1.1.1
kubernetes==10.0.1

Writing requirements.txt


## Install the packages listed in requirements.txt using pip

In [5]:
!pip install -r requirements.txt --user 

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

## Import Libraries

In [1]:
from kubernetes.client import V1PodTemplateSpec
from kubernetes.client import V1ObjectMeta
from kubernetes.client import V1PodSpec
from kubernetes.client import V1Container
from kubernetes.client import V1VolumeMount
from kubernetes.client import V1Volume
from kubernetes.client import V1PersistentVolumeClaimVolumeSource
from kubernetes.client import V1ResourceRequirements

from kubeflow.tfjob import constants
from kubeflow.tfjob import utils
from kubeflow.tfjob import V1ReplicaSpec
from kubeflow.tfjob import V1TFJob
from kubeflow.tfjob import V1TFJobSpec
from kubeflow.tfjob import TFJobClient


import time
import re, os
import tensorflow as tf
import pandas as pd
import numpy as np
import logging
import sys
import importlib

/home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

## Set up Kubeflow Fairing for training and predictions on On-premise
Import the fairing library and configure the onprem environment that your training or prediction job will run in.

In [2]:
from kubernetes import client as k8s_client
from kubernetes.client import rest as k8s_rest
from kubernetes import config as k8s_config
from kubernetes.client.rest import ApiException

from kubeflow import fairing   
from kubeflow.fairing import utils as fairing_utils
from kubeflow.fairing import TrainJob
from kubeflow.fairing.preprocessors.function import FunctionPreProcessor
from kubeflow.fairing.preprocessors import base as base_preprocessor
from kubeflow.fairing.builders.cluster.cluster import ClusterBuilder

from kubeflow.fairing.cloud.k8s import MinioUploader
from kubeflow.fairing.builders.cluster.minio_context import MinioContextSource
from kubeflow.fairing import PredictionEndpoint
from kubeflow.fairing.kubernetes.utils import mounting_pvc
from kubeflow.fairing.kubernetes.utils import mounting_pvc

BackendClass = getattr(importlib.import_module('kubeflow.fairing.backends'), "KubernetesBackend")
namespace = fairing_utils.get_current_k8s_namespace()
print("Namespace : %s"%namespace)

Namespace : anonymous


## Get minio-service cluster IP to upload docker build context
#### Set DOCKER_REGISTRY
The DOCKER_REGISTRY variable is used to push the newly built image. 
Please change the variable to the registry for which you've configured credentials.

In [3]:
DOCKER_REGISTRY = "docker.io/poornimadevii"

k8s_config.load_incluster_config()
api_client = k8s_client.CoreV1Api()
custom_api=k8s_client.CustomObjectsApi()
minio_service_endpoint = None

try:
    minio_service_endpoint = api_client.read_namespaced_service(name='minio-service', namespace='kubeflow').spec.cluster_ip
except ApiException as e:
    if e.status == 403:
        logging.warning(f"The service account doesn't have sufficient privileges "
                      f"to get the kubeflow minio-service. "
                      f"You will have to manually enter the minio cluster-ip. "
                      f"To make this function work ask someone with cluster "
                      f"priveleges to create an appropriate "
                      f"clusterrolebinding by running a command.\n"
                      f"kubectl create --namespace=kubeflow rolebinding "
                       "--clusterrole=kubeflow-view "
                       "--serviceaccount=${NAMESPACE}:default-editor "
                       "${NAMESPACE}-minio-view")
        logging.error("API access denied with reason: {e.reason}")

s3_endpoint = minio_service_endpoint
minio_endpoint = "http://"+s3_endpoint+":9000"
minio_username = "minio"
minio_key = "minio123"
minio_region = "us-east-1"
print(minio_endpoint)


minio_uploader = MinioUploader(endpoint_url=minio_endpoint, minio_secret=minio_username, minio_secret_key=minio_key, region_name=minio_region)
minio_context_source = MinioContextSource(endpoint_url=minio_endpoint, minio_secret=minio_username, minio_secret_key=minio_key, region_name=minio_region)

http://10.98.188.38:9000


### Create a config-map in the namespace you're using with the docker config

In [4]:
!kubectl create --namespace $namespace configmap docker-config --from-file=./config.json

configmap/docker-config created


## Update Dockerfile
Update dockerfile base image with tensorflow-gpu image if device type is GPU.

In [6]:
path="cisco-kubeflow-starter-pack/apps/networking/ble-localization/onprem"
dockerfile_path=os.path.join(path, "fairing/Dockerfile")
device_type="cpu"  #Provide cpu or gpu
if device_type=="gpu":
    !sed -i "s/py3/gpu/g" $dockerfile_path
    !cat $dockerfile_path
else:
    !cat $dockerfile_path

FROM tensorflow/tensorflow:1.15.2-py3
RUN pip install pandas
ADD blerssi-model.py  /opt/blerssi-model.py
ADD iBeacon_RSSI_Labeled.csv /opt/iBeacon_RSSI_Labeled.csv
RUN chmod +x /opt/blerssi-model.py  /opt/iBeacon_RSSI_Labeled.csv 
CMD ["python","/opt/blerssi-model.py"]


## Build Docker Image
Note: Upload dataset, Dockerfile and blerssi-model.py into notebook

In [7]:
output_map= {
    os.path.join(path, "fairing/Dockerfile"): "Dockerfile",
    os.path.join(path, "fairing/blerssi-model.py"):"blerssi-model.py",
    os.path.join(path, "data/iBeacon_RSSI_Labeled.csv"): "iBeacon_RSSI_Labeled.csv"
}
preprocessor = base_preprocessor.BasePreProcessor(output_map=output_map)

preprocessor.preprocess()
builder = ClusterBuilder(registry=DOCKER_REGISTRY, preprocessor=preprocessor, context_source=minio_context_source)

builder.build()

Building image using cluster builder.
Creating docker context: /tmp/fairing_context__1l_3yq6
/tmp/fairing_dockerfile_zwv5zv71 already exists in Fairing context, skipping...
Waiting for fairing-builder-dgm5j-qf6vf to start...
Waiting for fairing-builder-dgm5j-qf6vf to start...
Waiting for fairing-builder-dgm5j-qf6vf to start...
Waiting for fairing-builder-dgm5j-qf6vf to start...
Pod started running True


INFO[0001] Resolved base name tensorflow/tensorflow:1.15.2-py3 to tensorflow/tensorflow:1.15.2-py3
INFO[0001] Resolved base name tensorflow/tensorflow:1.15.2-py3 to tensorflow/tensorflow:1.15.2-py3
INFO[0001] Downloading base image tensorflow/tensorflow:1.15.2-py3
INFO[0002] Error while retrieving image from cache: getting file info: stat /cache/sha256:28b5f547969d70f825909c8fe06675ffc2959afe6079aeae754afa312f6417b9: no such file or directory
INFO[0002] Downloading base image tensorflow/tensorflow:1.15.2-py3
INFO[0002] Built cross stage deps: map[]
INFO[0002] Downloading base image tensorflow/tensorflow:1.15.2-py3
INFO[0003] Error while retrieving image from cache: getting file info: stat /cache/sha256:28b5f547969d70f825909c8fe06675ffc2959afe6079aeae754afa312f6417b9: no such file or directory
INFO[0003] Downloading base image tensorflow/tensorflow:1.15.2-py3
INFO[0004] Unpacking rootfs as cmd RUN pip install pandas requires it.
INFO[0022] Taking snapshot of full filesystem...
INFO[0031

In [8]:
builder.image_tag

'docker.io/poornimadevii/fairing-job:23DC69B'

## Create Katib Experiment

In [9]:
experiment={
  "apiVersion": "kubeflow.org/v1alpha3",
  "kind": "Experiment",
  "metadata": {
    "namespace": "anonymous",
    "labels": {
      "controller-tools.k8s.io": "1.0"
    },
    "name": "blerssi"
  },
  "spec": {
    "objective": {
      "type": "maximize",
      "goal": 0.5,
      "objectiveMetricName": "accuracy",
      "additionalMetricNames": [
        "Train-accuracy"
      ]
    },
    "algorithm": {
      "algorithmName": "random"
    },
    "parallelTrialCount": 5,
    "maxTrialCount": 5,
    "maxFailedTrialCount": 3,
    "parameters": [
      {
        "name": "--batch-size",
        "parameterType": "categorical",
        "feasibleSpace": {
          "list": [
            "16",
            "32",
            "48",
            "64"
          ]
        }
      },
      {
        "name": "--learning-rate",
        "parameterType": "double",
        "feasibleSpace": {
          "min": "0.01",
          "max": "0.03"
        }
      }
    ],
    "trialTemplate": {
      "goTemplate": {
        "rawTemplate": "apiVersion: batch/v1\nkind: Job\nmetadata:\n  name: {{.Trial}}\n  namespace: {{.NameSpace}}\nspec:\n  template:\n    spec:\n      containers:\n      - name: {{.Trial}}\n        image: %s\n        command:\n        - \"python\"\n        - \"/opt/blerssi-model.py\"\n        {{- with .HyperParameters}}\n        {{- range .}}\n        - \"{{.Name}}={{.Value}}\"\n        {{- end}}\n        {{- end}}\n      restartPolicy: Never"%builder.image_tag
      }
    }
  }
}

experiment_name=experiment["metadata"]["name"]
custom_api.create_namespaced_custom_object(group="kubeflow.org", version="v1alpha3", namespace=namespace, plural="experiments", body=experiment)

{'apiVersion': 'kubeflow.org/v1alpha3',
 'kind': 'Experiment',
 'metadata': {'creationTimestamp': '2020-05-08T07:40:44Z',
  'generation': 1,
  'labels': {'controller-tools.k8s.io': '1.0'},
  'name': 'blerssi',
  'namespace': 'anonymous',
  'resourceVersion': '6268870',
  'selfLink': '/apis/kubeflow.org/v1alpha3/namespaces/anonymous/experiments/blerssi',
  'uid': '9ddf9c29-98ec-41b0-92ae-024d878f6145'},
 'spec': {'algorithm': {'algorithmName': 'random'},
  'maxFailedTrialCount': 3,
  'maxTrialCount': 5,
  'metricsCollectorSpec': {'collector': {'kind': 'StdOut'}},
  'objective': {'additionalMetricNames': ['Train-accuracy'],
   'goal': 0.5,
   'objectiveMetricName': 'accuracy',
   'type': 'maximize'},
  'parallelTrialCount': 5,
  'parameters': [{'feasibleSpace': {'list': ['16', '32', '48', '64']},
    'name': '--batch-size',
    'parameterType': 'categorical'},
   {'feasibleSpace': {'max': '0.03', 'min': '0.01'},
    'name': '--learning-rate',
    'parameterType': 'double'}],
  'trialTemp

## Wait for experiment succeeded

In [10]:
status=False
while True:
    conditions=conditions=custom_api.get_namespaced_custom_object_status(group="kubeflow.org", version="v1alpha3", namespace=namespace, plural="experiments", name=experiment["metadata"]["name"])["status"]["conditions"]
    for i in range(len(conditions)):
        if (conditions[i]['type'])=='Succeeded':
            status=True
            print("Experiment Status: %s"%conditions[i]['type'])
            break
        
    if status:
        break
    print("Experiment Status: %s"%conditions[i]['type'])
    time.sleep(60)

Experiment Status: Created
Experiment Status: Created
Experiment Status: Running
Experiment Status: Succeeded


## Get Optimal Hyperparameters

In [11]:
tuning_params=custom_api.get_namespaced_custom_object("kubeflow.org", "v1alpha3", namespace, "experiments", experiment["metadata"]["name"])["status"]["currentOptimalTrial"]["parameterAssignments"]
tuning_params

[{'name': '--batch-size', 'value': '48'},
 {'name': '--learning-rate', 'value': '0.013594103715353231'}]

## Get PVC Name

In [12]:
pvcname = !(echo  $HOSTNAME | sed 's/.\{2\}$//')
pvc = "workspace-"+pvcname[0]
print(pvc)

workspace-teja


### Define TFJob Class to create training job

In [13]:
tfjob_name="blerssi-tfjob"
class Tfjob(object):

    def get_tfjob_params(self):
    
        #Defining a Volume Mount
        volume_mount = V1VolumeMount(name="fairing-volume", mount_path="/mnt/")

        #Defining a Persistent Volume Claim
        persistent_vol_claim = V1PersistentVolumeClaimVolumeSource(claim_name=pvc)

        #Defining a Volume
        volume = V1Volume(name="fairing-volume", persistent_volume_claim=persistent_vol_claim)
        
        if device_type=="gpu":
            #Defining a Container
            container = V1Container(
                name="tensorflow",            
                image=builder.image_tag,
                volume_mounts=[volume_mount],
                resources=V1ResourceRequirements(limits={"nvidia.com/gpu": 1}),
                command=["python"],
                args=[
                    "/opt/blerssi-model.py",
                    "--batch-size", "%s"%tuning_params[0]["value"],
                    "--learning-rate", "%s"%tuning_params[1]["value"],
                    "--tf-model-dir", "/mnt/model/",
                    "--tf-export-dir", "/mnt/model/"]
            )
        else:
            #Defining a Container
            container = V1Container(
                name="tensorflow",            
                image=builder.image_tag,
                volume_mounts=[volume_mount],
                command=["python"],
                args=[
                    "/opt/blerssi-model.py",
                    "--batch-size", "%s"%tuning_params[0]["value"],
                    "--learning-rate", "%s"%tuning_params[1]["value"],
                    "--tf-model-dir", "/mnt/model/",
                    "--tf-export-dir", "/mnt/model/"]
            )
        
        return (volume_mount, persistent_vol_claim, volume, container)
        
    def get_tfjob_nodes(self):
    
        params = self.get_tfjob_params()

        #Defining a Master
        master = V1ReplicaSpec(replicas=1,
                               restart_policy="Never",
                               template=V1PodTemplateSpec(spec=V1PodSpec(
                                                    containers=[params[3]],
                                                    volumes=[params[2]])))
        
        #Defining Worker Spec
        worker = V1ReplicaSpec(replicas=1,
                               restart_policy="Never",
                               template=V1PodTemplateSpec(spec=V1PodSpec(
                                                    containers=[params[3]],
                                                    volumes=[params[2]],
                                                    
                               )))
        
        #Defining Parameter server(PS) Spec
        ps = V1ReplicaSpec(replicas=1,
                               restart_policy="Never",
                               template=V1PodTemplateSpec(spec=V1PodSpec(
                                                    containers=[params[3]],
                                                    volumes=[params[2]])))
        
        return (master,worker,ps)
    
    def create_tfjob(self):
        
        tfjob_node_spec = self.get_tfjob_nodes()
        
        #Defining TFJob
        tfjob = V1TFJob(
            api_version="kubeflow.org/v1",
            kind="TFJob",
            metadata=V1ObjectMeta(name=tfjob_name,namespace=namespace),
            spec=V1TFJobSpec(
                clean_pod_policy="None",
                tf_replica_specs={"PS":tfjob_node_spec[2],"Worker": tfjob_node_spec[1],"Master":tfjob_node_spec[0]}
            )
        )
        
        #Creating TFJob
        tfjob_client = TFJobClient()
        tfjob_client.create(tfjob, namespace=namespace)

## Define Blerssi class to be used by Kubeflow fairing
## ( Must necessarily contain train() and predict() methods)

In [14]:
class BlerssiServe(object):
    
    def __init__(self):
         self.model = None
                 

    def train(self):
         Tfjob().create_tfjob()
        
    def predict(self,X,feature_names=None):
        
        global model_input1, model_input, predictor
        feature_col=["b3001", "b3002","b3003","b3004","b3005","b3006","b3007","b3008","b3009","b3010","b3011","b3012","b3013"]
        model_input1=tf.train.Example()
        for i in range(len(X)):
            model_input1.features.feature[feature_col[i]].float_list.value.append(X[i])
            
        path=os.path.join(os.getcwd(), "/mnt/model/")
        for dir in os.listdir(path):
            if re.match('[0-9]',dir):
                exported_path=os.path.join(path,dir)
                break
        
        # Open a Session to predict
        with tf.Session() as sess:
         tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], exported_path)
         model_input =model_input1
         
         predictor= tf.contrib.predictor.from_saved_model(exported_path,signature_def_key='predict')
         input_tensor=tf.get_default_graph().get_tensor_by_name("input_example_tensor:0")
            
         model_input=model_input.SerializeToString()
         output_dict= predictor({"examples":[model_input]})
        sess.close()
        
        response = output_dict.items()
        print(response)
        response1 = output_dict['class_ids']
        return response1

## Train an Blerssi model remotely on Kubeflow
Kubeflow Fairing packages the BlerssiServe class, the training data, and the training job's software prerequisites as a Docker image. Then Kubeflow Fairing deploys and runs the training job on kubeflow.

In [15]:
train_job = TrainJob(BlerssiServe, input_files=[os.path.join(path,"data/iBeacon_RSSI_Labeled.csv"), "requirements.txt"],
                     pod_spec_mutators = [mounting_pvc(pvc_name=pvc, pvc_mount_path="/mnt/")],
                     docker_registry=DOCKER_REGISTRY, backend=BackendClass(build_context_source=minio_context_source))
train_job.submit()

Using default base docker image: registry.hub.docker.com/library/python:3.6.9
Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
Building the docker image.
Building image using cluster builder.
/home/jovyan/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Creating docker context: /tmp/fairing_context_f8nkrb00
/home/jovyan/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Waiting for fairing-builder-mf5l4-m9zpp to start...
Waiting for fairing-builder-mf5l4-m9zpp to start...
Waiting for fairing-builder-mf5l4-m9zpp to start...
Waiting for fairing-builder-mf5l4-m9zpp to start...
Pod started running True


INFO[0001] Resolved base name registry.hub.docker.com/library/python:3.6.9 to registry.hub.docker.com/library/python:3.6.9
INFO[0001] Resolved base name registry.hub.docker.com/library/python:3.6.9 to registry.hub.docker.com/library/python:3.6.9
INFO[0001] Downloading base image registry.hub.docker.com/library/python:3.6.9
INFO[0002] Error while retrieving image from cache: getting file info: stat /cache/sha256:036d4ab50fa49df89e746cf1b5369c88db46e8af2fbd08531788e7d920e9a491: no such file or directory
INFO[0002] Downloading base image registry.hub.docker.com/library/python:3.6.9
INFO[0003] Built cross stage deps: map[]
INFO[0003] Downloading base image registry.hub.docker.com/library/python:3.6.9
INFO[0003] Error while retrieving image from cache: getting file info: stat /cache/sha256:036d4ab50fa49df89e746cf1b5369c88db46e8af2fbd08531788e7d920e9a491: no such file or directory
INFO[0003] Downloading base image registry.hub.docker.com/library/python:3.6.9
INFO[0004] Unpacking rootfs as cm

The job fairing-job-kfhct launched.
Waiting for fairing-job-kfhct-qm59q to start...
Waiting for fairing-job-kfhct-qm59q to start...
Waiting for fairing-job-kfhct-qm59q to start...
Waiting for fairing-job-kfhct-qm59q to start...
Waiting for fairing-job-kfhct-qm59q to start...
Pod started running True


/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Cleaning up job fairing-job-kfhct...


'fairing-job-kfhct'

## Deploy the trained model to Kubeflow for predictions
Kubeflow Fairing packages the BlerssiServe class, the trained model, and the prediction endpoint's software prerequisites as a Docker image. Then Kubeflow Fairing deploys and runs the prediction endpoint on Kubeflow.

In [21]:
endpoint = PredictionEndpoint(BlerssiServe, input_files=[os.path.join(path,"data/iBeacon_RSSI_Labeled.csv"),"requirements.txt"],
                              docker_registry=DOCKER_REGISTRY,
                              pod_spec_mutators = [mounting_pvc(pvc_name=pvc, pvc_mount_path="/mnt/")],
                              backend=BackendClass(build_context_source=minio_context_source))
endpoint.create()

Using default base docker image: registry.hub.docker.com/library/python:3.6.9
Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
Building the docker image.
Building image using cluster builder.
/home/jovyan/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Creating docker context: /tmp/fairing_context_6m0bqtef
/home/jovyan/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Waiting for fairing-builder-2zj9r-8ttd8 to start...
Waiting for fairing-builder-2zj9r-8ttd8 to start...
Waiting for fairing-builder-2zj9r-8ttd8 to start...
Waiting for fairing-builder-2zj9r-8ttd8 to start...
Pod started running True


INFO[0001] Resolved base name registry.hub.docker.com/library/python:3.6.9 to registry.hub.docker.com/library/python:3.6.9
INFO[0001] Resolved base name registry.hub.docker.com/library/python:3.6.9 to registry.hub.docker.com/library/python:3.6.9
INFO[0001] Downloading base image registry.hub.docker.com/library/python:3.6.9
INFO[0002] Error while retrieving image from cache: getting file info: stat /cache/sha256:036d4ab50fa49df89e746cf1b5369c88db46e8af2fbd08531788e7d920e9a491: no such file or directory
INFO[0002] Downloading base image registry.hub.docker.com/library/python:3.6.9
INFO[0003] Built cross stage deps: map[]
INFO[0003] Downloading base image registry.hub.docker.com/library/python:3.6.9
INFO[0003] Error while retrieving image from cache: getting file info: stat /cache/sha256:036d4ab50fa49df89e746cf1b5369c88db46e8af2fbd08531788e7d920e9a491: no such file or directory
INFO[0003] Downloading base image registry.hub.docker.com/library/python:3.6.9
INFO[0004] Unpacking rootfs as cm

Deploying the endpoint.
Cluster endpoint: http://fairing-service-7rv29.anonymous.svc.cluster.local:5000/predict
Prediction endpoint: http://fairing-service-7rv29.anonymous.svc.cluster.local:5000/predict


## Wait for  prediction pod ready state

In [22]:
!kubectl get deploy -l fairing-deployer=serving -n anonymous

NAME                     READY   UP-TO-DATE   AVAILABLE   AGE
fairing-deployer-9wq6z   1/1     1            1           2m47s


##  Get prediction endpoint

In [23]:
endpoint.url

'http://fairing-service-7rv29.anonymous.svc.cluster.local:5000/predict'

## Call the prediction endpoint
Use the endpoint from previous cell

In [24]:
! curl -v http://fairing-service-7rv29.anonymous.svc.cluster.local:5000/predict -H "Content-Type: application/x-www-form-urlencoded" -d 'json={"data":{"ndarray":[-0.458086, -0.6244, 2.354243, -0.404581, 1.421444, 1.767642, 2.637829, -0.603085, 0.382779, -0.378999, -0.341798, -0.303249, -0.327776]}}'

*   Trying 10.101.75.89...
* TCP_NODELAY set
* Connected to fairing-service-7rv29.anonymous.svc.cluster.local (10.101.75.89) port 5000 (#0)
> POST /predict HTTP/1.1
> Host: fairing-service-7rv29.anonymous.svc.cluster.local:5000
> User-Agent: curl/7.58.0
> Accept: */*
> Content-Type: application/x-www-form-urlencoded
> Content-Length: 162
> 
* upload completely sent off: 162 out of 162 bytes
* HTTP 1.0, assume close after body
< HTTP/1.0 200 OK
< Content-Type: application/json
< Content-Length: 54
< Access-Control-Allow-Origin: *
< Server: Werkzeug/1.0.1 Python/3.6.9
< Date: Fri, 08 May 2020 08:47:37 GMT
< 
{"data":{"names":["t:0"],"ndarray":[[69]]},"meta":{}}
* Closing connection 0


## Clean up the prediction endpoint
Delete the prediction endpoint created by this notebook.

In [25]:
endpoint.delete()

Deleting the endpoint. 
Deleted service: anonymous/fairing-service-7rv29
Deleted deployment: anonymous/fairing-deployer-9wq6z


## Clean up the Katib Experiment
Delete the katib experiment created by this notebook.

In [26]:
!kubectl delete experiment $experiment_name -n $namespace

experiment.kubeflow.org "blerssi" deleted


## Clean up the TFjob
Delete the TFjob created by this notebook.

In [28]:
TFJobClient().delete(tfjob_name, namespace=namespace)

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'blerssi-tfjob',
  'group': 'kubeflow.org',
  'kind': 'tfjobs',
  'uid': 'b6a3dae2-c00a-4914-8d29-7d1ccbb545b9'}}

## Delete config.json and requirements.txt

In [29]:
!rm -rf config.json requirements.txt
if device_type=="gpu":
    !sed -i "s/gpu/py3/g" $dockerfile_path
    !cat $dockerfile_path